In [46]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix,accuracy_score,recall_score,precision_score,f1_score,roc_auc_score,classification_report
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import RepeatedStratifiedKFold,RepeatedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import StackingClassifier,StackingRegressor
from matplotlib import pyplot
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso, Ridge, ElasticNet,RidgeCV
from sklearn.svm import LinearSVR

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import MinMaxScaler

import warnings
 

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import warnings
warnings.filterwarnings("ignore")

In [4]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
train_data["source"] = "train"
test_data["source"] = "test"

In [5]:
df = pd.concat([train_data,test_data])

In [6]:
df.isna().sum()

mass_npea             0
size_npear            0
malign_ratio          0
damage_size           0
exposed_area          0
std_dev_malign        0
err_malign            0
malign_penalty        0
damage_ratio          0
tumor_size        36584
source                0
dtype: int64

In [7]:
df.head(10)

,mass_npea,size_npear,malign_ratio,damage_size,exposed_area,std_dev_malign,err_malign,malign_penalty,damage_ratio,tumor_size,source
0,6930.90,2919.02,0.42116,51.8298,9.888294e+05,109.4870,2758.76,72,39.3620,14.103,train
1,15635.70,4879.36,0.31206,223.5500,2.058426e+06,248.8810,5952.53,240,22.0253,2.648,train
2,10376.20,2613.88,0.25191,127.3370,1.434676e+06,160.0930,4635.26,73,29.9963,1.688,train
3,13093.80,4510.06,0.34444,155.4400,1.812195e+06,173.0150,5273.87,32,28.1354,3.796,train
4,7545.21,2882.36,0.38201,85.1237,1.043918e+06,124.4140,3263.35,57,35.0200,18.023,train
5,6851.09,2195.18,0.32041,72.8283,9.484467e+05,97.1881,3688.57,40,36.3481,1.709,train
6,7069.24,1886.09,0.26680,58.2686,1.024783e+06,76.9447,3168.44,22,39.9961,9.937,train
7,16446.10,5115.45,0.31104,204.9740,2.167355e+06,265.8810,6425.91,242,22.9533,2.510,train
8,6814.73,2043.78,0.29990,90.0889,9.669835e+05,99.6286,3428.54,27,37.1642,12.568,train
9,5049.75,949.82,0.18809,41.2957,6.906229e+05,70.4142,2734.59,27,41.1366,13.428,train


In [8]:
train = df[df["source"]=="train"]
test  = df[df["source"]=="test"]
del train["source"]
del test["source"]

In [9]:
train_X = train.drop(columns='tumor_size')
train_Y = train["tumor_size"]
test_X = test.drop(columns='tumor_size')
test_Y = test["tumor_size"]

In [19]:
def run_model(model, train_X, train_y, test_X, sample, filename ):
    model.fit(train_X, train_y)
   
    pred_train = model.predict(train_X)
    pred_test = model.predict(test_X)
    pred1 = pred_test
    print("RMSE of Train Data = ",np.sqrt(mean_squared_error(train_y, pred_train)))
    score = cross_val_score(model, train_X, train_y, cv = 10, scoring = "neg_root_mean_squared_error" )
    print("Cross validation Score  = ",np.mean(np.abs(score)))
   
    sample["tumor_size"] = pred_test.astype(int)
    test_y = pred_test
    sample.to_csv(filename, index = False)

In [20]:
sample=pd.read_csv("sample_submission.csv")

In [21]:
sample.head()

,tumor_size
0,3.418762
1,7.348026
2,3.096391
3,10.489805
4,27.416072


In [22]:
model = LinearRegression()
run_model(model, train_X, train_Y, test_X, sample, "test_result_Linear_regression_df.csv" )

RMSE of Train Data =  5.131363676156946
Cross validation Score  =  5.143827372331534


In [17]:
def run_model(model, train_X, train_y, test_X, sample, filename ):
    model.fit(train_X, train_y)
    pred_train = model.predict(train_X)
    pred_test = model.predict(test_X)
    pred_test = abs(pred_test)
    print("RMSE of Train Data = ",np.sqrt(mean_squared_error(train_y, pred_train)))
    score = cross_val_score(model, train_X, train_y, cv = 10, scoring = "neg_root_mean_squared_error" )
    print("Cross validation Score  = ",np.mean(np.abs(score)))
    sample["tumor_size"] = pred_test
    sample.to_csv(filename, index = False)

In [18]:
model = RandomForestRegressor()
run_model(model, train_X, train_Y, test_X, sample, "test_result_RandomForest_regression.csv")

RMSE of Train Data =  1.4857247760453653
Cross validation Score  =  4.019286380295858


In [41]:
parameters = {'n_estimators':(0,25,50,75,100,150,200,225,250) }
model = RandomForestRegressor(n_jobs=-1)
clf = GridSearchCV(model, parameters, verbose = 2)
#clf = RandomizedSearchCV(model, parameters, verbose = 1, cv=10)
clf.fit(train_X, train_Y)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] n_estimators=0 ..................................................
[CV] ................................... n_estimators=0, total=   0.0s
[CV] n_estimators=0 ..................................................
[CV] ................................... n_estimators=0, total=   0.0s
[CV] n_estimators=0 ..................................................
[CV] ................................... n_estimators=0, total=   0.0s
[CV] n_estimators=0 ..................................................
[CV] ................................... n_estimators=0, total=   0.0s
[CV] n_estimators=0 ..................................................
[CV] ................................... n_estimators=0, total=   0.0s
[CV] n_estimators=25 .................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] .................................. n_estimators=25, total=   9.5s
[CV] n_estimators=25 .................................................
[CV] .................................. n_estimators=25, total=   2.8s
[CV] n_estimators=25 .................................................
[CV] .................................. n_estimators=25, total=   2.8s
[CV] n_estimators=25 .................................................
[CV] .................................. n_estimators=25, total=   2.8s
[CV] n_estimators=25 .................................................
[CV] .................................. n_estimators=25, total=   2.7s
[CV] n_estimators=50 .................................................
[CV] .................................. n_estimators=50, total=   4.1s
[CV] n_estimators=50 .................................................
[CV] .................................. n_estimators=50, total=   4.2s
[CV] n_estimators=50 .................................................
[CV] .

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  6.1min finished


GridSearchCV(estimator=RandomForestRegressor(n_jobs=-1),
             param_grid={'n_estimators': (0, 25, 50, 75, 100, 150, 200, 225,
                                          250)},
             verbose=2)

In [42]:
clf.best_estimator_

RandomForestRegressor(n_estimators=200, n_jobs=-1)

In [43]:
model = RandomForestRegressor(n_estimators=200, n_jobs=-1)
run_model(model, train_X, train_Y, test_X, sample, "test_result_RandomForest_GridSearch.csv")

RMSE of Train Data =  1.4727105616826164
Cross validation Score  =  4.010100741418396


In [27]:
model.fit(train_X, train_Y)
pd.Series(model.feature_importances_, index=train_X.columns).sort_values(ascending=False)

malign_ratio      0.211476
damage_size       0.175003
malign_penalty    0.129544
std_dev_malign    0.098621
err_malign        0.096115
exposed_area      0.081523
damage_ratio      0.077440
size_npear        0.074450
mass_npea         0.055828
dtype: float64

In [28]:
scaler = MinMaxScaler()

scaler.fit(train_X)
train_x = scaler.transform(train_X)
scaler.fit(test_X)
test_x = scaler.transform(test_X)

In [29]:
modlel =XGBRegressor()
run_model(model, train_x, train_Y, test_x, sample, "test_result_XGB.csv")

RMSE of Train Data =  1.4662838087529513
Cross validation Score  =  4.0047355880668905


In [60]:
level0 = list()
level0.append(('RF', RandomForestRegressor()))
level0.append(('knn', KNeighborsRegressor()))
level0.append(('cart', DecisionTreeRegressor()))
level0.append(('SVM',LinearSVR()))
level0.append(('Ridge', RidgeCV()))
level1 = RandomForestRegressor()
model = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)
model.fit(train_X, train_Y)
pred_train = model.predict(train_X)
pred_test = model.predict(test_X)
#pred_test = abs(pred_test)
sample["tumor_size"] = pred_test
sample.to_csv("stacking_regression_3.csv", index = False)
print("RMSE of Train Data = ",np.sqrt(mean_squared_error(train_Y, pred_train)))
score = cross_val_score(model, train_X, train_Y, cv = 10, scoring = "neg_root_mean_squared_error" )
print("Cross validation Score  = ",np.mean(np.abs(score)))

RMSE of Train Data =  1.6675869242380708
Cross validation Score  =  4.167821459433607


In [61]:
df.head(5)

,mass_npea,size_npear,malign_ratio,damage_size,exposed_area,std_dev_malign,err_malign,malign_penalty,damage_ratio,tumor_size,source
0,6930.90,2919.02,0.42116,51.8298,9.888294e+05,109.487,2758.76,72,39.3620,14.103,train
1,15635.70,4879.36,0.31206,223.5500,2.058426e+06,248.881,5952.53,240,22.0253,2.648,train
2,10376.20,2613.88,0.25191,127.3370,1.434676e+06,160.093,4635.26,73,29.9963,1.688,train
3,13093.80,4510.06,0.34444,155.4400,1.812195e+06,173.015,5273.87,32,28.1354,3.796,train
4,7545.21,2882.36,0.38201,85.1237,1.043918e+06,124.414,3263.35,57,35.0200,18.023,train


In [64]:
train_X = train.drop(columns=['tumor_size','mass_npea'])
train_Y = train["tumor_size"]
test_X = test.drop(columns=['tumor_size','mass_npea'])
test_Y = test["tumor_size"]

In [65]:
model = RandomForestRegressor()
run_model(model, train_X, train_Y, test_X, sample, "test_result_RandomForest_GridSearch.csv")

RMSE of Train Data =  1.4923777563246776
Cross validation Score  =  4.027879116554126
